In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [3]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib

STITCH_BATCH_SIZE = 64
RESULT_NAME = "result"

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [ ]:
scoreMapper = ScoreMapper(nets, data_score)
generator = generate_networks(nets, scoreMapper, data_score, threshold=0.9, totalThreshold=0.8)
    
with Report(STITCH_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'w') as report:
    for i,(s,net) in enumerate(generator):
        netname = f"_results/{RESULT_NAME}/net{i:03}"
        report.evaluate(nets, net, netname, s, dataset_val)   

100%|██████████| 7/7 [00:19<00:00,  2.78s/it]


potential next fragments: 63
potential next fragments after thresholding of 0.9: 1
totalscore 1.0000001192092896


100%|██████████| 7/7 [00:07<00:00,  1.11s/it]


potential next fragments: 63
potential next fragments after thresholding of 0.9: 1
totalscore 0.9999999999999858


100%|██████████| 7/7 [00:08<00:00,  1.17s/it]


...

100%|██████████| 7/7 [00:37<00:00,  5.41s/it]


potential next fragments: 63
potential next fragments after thresholding of 0.9: 1
totalscore 1.0000001192092896


100%|██████████| 7/7 [01:48<00:00, 15.47s/it]


potential next fragments: 63
potential next fragments after thresholding of 0.9: 1
totalscore 0.9999999999999858


 57%|█████▋    | 4/7 [01:25<00:55, 18.49s/it]

In [51]:
# nets[0][0].fragment.graph.input

In [49]:
# from stitchnet.stitchonnx.utils import list_net_ops
# list_net_ops(nets[0])